# Sample Implementation of AES in CBC-mode
Secret 1: I don't like vegetables\
Password: Unhealthy\
Ciphertext: b00d9978726acf0978c470b84de0a758f3adc082e4a7bd9159dce3a66f2d613758e9228b581e980c50a274608a259ffc0931cd7a6532d99ecd6594ce1dfc471f

Encrypted Secret 2: 7e55185da4f2c9589a571bac38ba0c8164bb8dd8e1bacba0bd99166123d3b0de8e455827fa56c7378a55cc1256a5ffbe3a8140e3ed2f9f276049ee28635b477d

In [14]:
import os
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

def generate_key(password, salt):
    #Generate key using PBKDF2-HMAC algo with SHA-256 hashing function (salt included)
    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=16,
        salt=salt,
        iterations=100000,
        backend=default_backend()
    )
    return kdf.derive(password.encode())

def encrypt(secret, password):
    #AES in CBC-mode
    #Generate salt
    salt = os.urandom(16)
    key = generate_key(password, salt)
    padder = padding.PKCS7(algorithms.AES.block_size).padder()
    padded_data = padder.update(secret.encode()) + padder.finalize()
    #Generate random initialisation vector
    init_vector = os.urandom(16)
    cipher = Cipher(algorithms.AES(key), modes.CBC(init_vector), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()
    #concat encrypted message + salt + initialisation vector
    return ciphertext + salt + init_vector

def decrypt(encrypted_data, password):
    #Extract ciphertext,salt and iv according to index
    ciphertext = encrypted_data[:-(16+16)] 
    salt = encrypted_data[-(16+16):-16]
    init_vector = encrypted_data[-16:]
    #regenerate the same key used to encrypt
    key = generate_key(password, salt)
    cipher = Cipher(algorithms.AES(key), modes.CBC(init_vector), backend=default_backend())
    decryptor = cipher.decryptor()
    decrypted_data = decryptor.update(ciphertext) + decryptor.finalize()
    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    return unpadder.update(decrypted_data) + unpadder.finalize()

def main():
    mode = input("Choose mode: (1) Encrypt (2) Decrypt: ")
    if mode == '1':
        secret = input("Enter your secret: ")
        password = input("Enter your password: ")
        encrypted_data = encrypt(secret, password)
        #Converting to hexadecimal format makes it easier to store/copy
        print("Encrypted data (in hex):", encrypted_data.hex())
    elif mode == '2':
        encrypted_data_hex = input("Enter the encrypted data (hex): ")
        #Convert the data from hexadecimal to bytes
        encrypted_data = bytes.fromhex(encrypted_data_hex)
        password = input("Enter your password: ")
        decrypted_secret = decrypt(encrypted_data, password)
        print("Secret Message:", decrypted_secret.decode())
    else:
        print("Invalid Input")

if __name__ == "__main__":
    main()

Choose mode: (1) Encrypt (2) Decrypt: 2
Enter the encrypted data (hex): b00d9978726acf0978c470b84de0a758f3adc082e4a7bd9159dce3a66f2d613758e9228b581e980c50a274608a259ffc0931cd7a6532d99ecd6594ce1dfc471f
Enter your password: Unhealthy
Secret Message: I don't like vegetables
